In [1]:
import pandas as pd
import numpy as np
import json
from tqdm.auto import tqdm
    
df = pd.read_csv("../../data/trade_imports.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)
df.sample(n=5)

,Index,to_country,from_country,Item,Year,Quantity,Unit,Coordinates
290914,411158,Sweden,"China, mainland",Food wastes,2016,2,Tonnes,"{'to_country': [18.643501, 60.128161], 'from_c..."
56124,80778,"China, Hong Kong SAR",Japan,Bread,2016,344,Tonnes,"{'to_country': [114.10949699999999, 22.396428]..."
268194,377000,Sierra Leone,France,"Milk, whole fresh cow",2016,14,Tonnes,"{'to_country': [-11.779889, 8.460555000000001]..."
164605,227854,Kazakhstan,Turkmenistan,"Onions, dry",2016,208,Tonnes,"{'to_country': [66.923684, 48.019573], 'from_c..."
208998,292761,Netherlands,Slovenia,Apples,2016,1,Tonnes,"{'to_country': [5.291266, 52.132633], 'from_co..."


In [2]:
# calculate the set of available countries
countrySet = sorted(set(df["to_country"]))

# get the set of produce used in the previous visualizations
previousDf = pd.read_csv("../../data/imp_exp_prod_flat_all.csv")
previousProduceSet = sorted(set(previousDf["Item"]))

In [3]:
def to_dictionary(row):
    dictionary = row.to_dict()
    dictionary["Coordinates"] = eval(dictionary["Coordinates"])
    return dictionary
    
# create an object containing all relevant data + top 10 countries that each country imports stuff from
importObject = {}
importObject["countries"] = countrySet

tqdm.pandas()
for country in tqdm(countrySet):
    countryObject = importObject[country] = {}

    # record the real top 10 countries (even if their goods were not in the previous visualizations)
    selected = df[df["to_country"] == country]
    countryObject["all_goods"] = list(selected.nlargest(10, "Quantity").apply(to_dictionary, axis=1))
    
    # then, only record top 10 countries whose crops/goods were part of the previous visualizations
    selected = selected[(selected["Item"]).isin(previousProduceSet)]
    countryObject["previous_goods_only"] = list(selected.nlargest(10, "Quantity").apply(to_dictionary, axis=1))

c:\users\roshini\appdata\local\programs\python\python37-32\lib\site-packages\tqdm\_tqdm.py:603: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
json.dump(importObject, open("../../data/trade_top_imports.json", "w"))